In [1]:
import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from scipy.stats import multivariate_normal # MVN not univariate

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import time

np.random.seed(7)


np.set_printoptions(suppress=True)

# Load in data files
file_path = "data.csv"
df = pd.read_csv(file_path, index_col='year')

file_path = "tracks_features.csv"
df2 = pd.read_csv(file_path, index_col='year')

# file_path = "archive\songs_normalize.csv"
# df3 = pd.read_csv(file_path, index_col='year')

# Delete unwanted/unnecessary columns
df.drop(['artists','id', 'name', 'release_date','popularity' ], axis = 1, inplace = True)
df2.drop(['artists','id', 'name', 'release_date', 'album_id','artist_ids', 'time_signature',\
          'track_number', 'disc_number', 'album' ], axis = 1, inplace = True)

df = pd.concat([df,df2])


### Only look at decades from 1950s to 1010s (2020 not included)
l_drop = np.arange(1921,1950)
l_drop = np.append(l_drop,2020)
drop_vals = np.array([0,1900,1908,1909,1917,1920])     #for df2
l_drop = np.concatenate((l_drop,drop_vals))

# Delete unwanted years
df.drop(labels=l_drop, axis=0, inplace = True)

enc = LabelEncoder()
labels = df.index
standardized_labels = np.array(labels)
enc.fit(df.index.unique())



lmao = df.index

# Create labels by each individual year
y = enc.transform(standardized_labels)
Y = enc.transform(np.unique(standardized_labels))

# Convert labels (year) to decades
y_decade = y//10
Y_decade = np.unique(y_decade)

enc.fit(df['explicit'].unique())
df['explicit'] = enc.transform(df['explicit'])


df.set_index(y_decade, inplace=True)

aa = df.index.value_counts().sort_index().to_numpy()
priors = aa/len(df.index)
check = np.sum(priors)
class_priors = np.diag(priors)
num_classes = len(Y_decade)
# print(df)

print()
N = len(df)
print('Labels:',Y_decade)
print('Total Datapoints:',N)
print("Counts Per Class:")
print(df.index.value_counts().sort_index())
df.head()


Labels: [0 1 2 3 4 5 6]
Total Datapoints: 1272462
Counts Per Class:
0     23109
1     28784
2     37183
3     48595
4    173049
5    443753
6    517989
dtype: int64


,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,0.782,0.633,106471,0.2610,1,0.0000,1,0.235,-16.389,1,0.7970,167.679,0.655
0,0.988,0.420,232933,0.0909,0,0.7860,10,0.104,-19.388,1,0.0409,123.089,0.227
0,0.993,0.394,177981,0.2580,0,0.0770,5,0.153,-9.779,0,0.1100,74.761,0.340
0,0.730,0.618,125300,0.2720,1,0.0000,6,0.146,-18.515,1,0.7310,67.141,0.449
0,0.993,0.475,188600,0.4070,0,0.0134,9,0.116,-13.011,1,0.0492,74.130,0.594


In [2]:
def regularized_cov(X, lambda_reg):
    n = X.shape[0]
    sigma = np.cov(X)
    # Selecting the regularization parameter should be performed using CV and a separate data subset
    # As I only went by training set performance (overfitting) in this problem, I settled on lambda=1/n
    sigma += lambda_reg * np.eye(n)
    return sigma

# Normalize data as zero mean gaussian and compute mean and stdev
covariance = df.std()
mean = df.mean()
X = (df-df.mean())/df.std()
# print(X)
mu = X.groupby([X.index]).mean().to_numpy()
n = mu.shape[1]
Sigma = np.array([regularized_cov(X[y_decade == l].T,(1/n)) for l in range(num_classes)])
# Sigma = np.array([np.cov(X[y_decade == l].T) for l in range(num_classes)])
# print(mu)
# print(Sigma)


In [3]:
### Calculate MLE

C = len(priors)
class_cond_likelihoods = np.array([multivariate_normal.pdf(X, mu[j], Sigma[j]) for j in range(C)])
# print(np.max(class_cond_likelihoods))

# Class Posterior
# P(yj | x) = p(x | yj) * P(yj) / p(x)
class_posteriors = class_priors.dot(class_cond_likelihoods)

decisions = np.argmax(class_posteriors, axis=0)

sample_class_counts = np.array([sum(y == j) for j in Y_decade])

# Confusion Matrix
conf_mat = np.zeros((C, C))
display_mat = np.zeros((C,C))
for i in range(C): # Each decision option
    for j in range(C): # Each class label
        ind_ij = np.argwhere((decisions==Y_decade[i]) & (y_decade==Y_decade[j]))
        display_mat[i, j] = len(ind_ij) # Average over class sample count
        conf_mat[i, j] = len(ind_ij)/sample_class_counts[j]

print("Total Number of Samples:",np.sum(display_mat))
print("Confusion matrix:")
print(display_mat.astype(int))
# print(1950+(Y_decade*10))

correct_class_samples = np.sum(np.diag(display_mat))
print("Total Mumber of Misclassified Samples: {}".format(N - correct_class_samples))

prob_error = 1 - (correct_class_samples / N)
print("Empirically Estimated Probability of Error: {:.4f}".format(prob_error))
print("Accuracy:",1-prob_error)

Total Number of Samples: 1272462.0
Confusion matrix:
[[  1194    453    396    457   2708   7183   5409]
 [  2227   3087   1298    734   3319   8127   5721]
 [   927   2914   7341   7837  13729  23068  20787]
 [    14     22    254    828    571    579    470]
 [   977   1029    851   2481  17242  22931  22067]
 [ 16305  19566  24178  30310 106524 284366 286614]
 [  1465   1713   2865   5948  28956  97499 176921]]
Total Mumber of Misclassified Samples: 781483.0
Empirically Estimated Probability of Error: 0.6142
Accuracy: 0.38584963637421


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Split data into test and train data
X_train, X_test, y_train, y_test = train_test_split(X.values, y_decade, test_size = 0.30)


start = time.time()

# # Random Forest Model
# clf = RandomForestClassifier(n_estimators = 100)
# print('done')

# clf.fit(X_train, y_train)
# print('done')

# feature_imp = pd.Series(clf.feature_importances_, index = df.columns).sort_values(ascending = False)
# print('done')

# print(feature_imp)


# y_pred = clf.predict(X_test)
 
# # metrics are used to find accuracy or error
# from sklearn import metrics 
# print()
 
# # using metrics module for accuracy calculation
# print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))



stop = time.time()
print()
print('Time to run (sec): ', stop - start) 
print('Time to run (min): ', (stop - start)/60) 


Time to run (sec):  0.0
Time to run (min):  0.0


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# Utility to visualize PyTorch network and shapes
from torchsummary import summary
from sklearn.model_selection import KFold # Important new include
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

torch.manual_seed(7) 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.cuda.device(device)

cuda


In [11]:
class TwoLayerMLP(nn.Module):
    # Two-layer MLP (not really a perceptron activation function...) network class
    
    def __init__(self, input_dim, hidden_dim, C):
        super(TwoLayerMLP, self).__init__()
        # Fully connected layer WX + b mapping from input_dim (n) -> hidden_layer_dim
        self.input_fc = nn.Linear(input_dim, hidden_dim)
        # Hidden Layer
        X = nn.Linear(hidden_dim, hidden_dim)

        # Output layer again fully connected mapping from hidden_layer_dim -> outputs_dim (C)
        self.output_fc = nn.Linear(hidden_dim, C)
        # Log Softmax (faster and better than straight Softmax)
        # dim=1 refers to the dimension along which the softmax operation is computed
        # In this case computing probabilities across dim 1, i.e., along classes at output layer
        self.log_softmax = nn.LogSoftmax(dim=1) 
        
    # Don't call this function directly!! 
    # Simply pass input to model and forward(input) returns output, e.g. model(X)
    def forward(self, X):
        # X = [batch_size, input_dim (n)]
        X = self.input_fc(X)
        # Non-linear activation function, e.g. ReLU (default good choice)
        # Could also choose F.softplus(x) for smooth-ReLU, empirically worse than ReLU
        X = F.relu(X)
        X = self.hidden_layer(X)
        # X = [batch_size, hidden_dim]
        
        # Add another hidden layer
        X = F.relu(X)
        X = self.hidden_layer(X)
        
        # Connect to last layer and output 'logits'
        X = self.output_fc(X)
        # Squash logits to probabilities that sum up to 1
        y = self.log_softmax(X)
        return y


In [7]:
def model_train(model, data, labels, criterion, optimizer, num_epochs=25):

    # Apparently good practice to set this "flag" too before training
    # Does things like make sure Dropout layers are active, gradients are updated, etc.
    # Probably not a big deal for our toy network, but still worth developing good practice
#     model.to(device)
#     data = data.to(device)
#     print(data.device)
    
    model.train()
    # Optimize the neural network
    for epoch in range(num_epochs):
        # These outputs represent the model's predicted probabilities for each class. 
        outputs = model(data)
        # Criterion computes the cross entropy loss between input and target
        loss = criterion(outputs, labels)
        # Set gradient buffers to zero explicitly before backprop
        optimizer.zero_grad()
        # Backward pass to compute the gradients through the network
        loss.backward()
        # GD step update
        optimizer.step()
        
    return model
    
    
def model_predict(model, data, labels):
    # Similar idea to model.train(), set a flag to let network know your in "inference" mode
#     model.to(device)
    labels.to(device)

    model.eval()
    N = len(data)
#     print(N)
    # Disabling gradient calculation is useful for inference, only forward pass!!
    with torch.no_grad():
        # Evaluate nn on test data and compare to true labels
        predicted_labels = model(data)
        # Back to numpy
        predicted_labels = predicted_labels.detach().cpu().numpy()
        Z = np.argmax(predicted_labels, 1)
#     print(labels)
    conf_mat = confusion_matrix(Z, labels.cpu())
    correct_class_samples = np.sum(np.diag(conf_mat))
    prob_error = 1 - (correct_class_samples / N)
#     print(conf_mat)
#     print("Total Number of Misclassified Samples: {:d}".format(N - correct_class_samples))
#     print("Empirically Estimated Probability of Error: {:.4f}".format(prob_error))
#     print()
    return Z, prob_error, conf_mat

# Z, acc = model_predict(model,X_tensor[0],labels[0])
# print(acc)

def neuron_cross_validation(X,labels,X_tensor,l_tensor):#,Xtest,labelsTest):

    K = 10   #No. of K-folds in CV
    kf = KFold(n_splits=K, shuffle=True)
    start = 1    # starting no. of neurons
    trials = 100      # no. of trials
    neurons = np.arange(start, start+trials, 1)
    n_hidden = len(neurons)
    
#     X = X.to(device)
#     labels = labels.to(device)

    #Assign data to GPU
    X_tensor = X_tensor.to(device)
    l_tensor = l_tensor.to(device)
    
    # Store predictions per degree using ordered X samples for plotting best fit lines
#     y_train_preds_ordered = np.empty(n_degs, dtype=np.ndarray)
    # Allocate space for CV
    # No need for training loss storage too but useful comparison
    mse_valid_mk = np.empty((n_hidden, K)) 
    mse_train_mk = np.empty((n_hidden, K)) # Indexed by model m, data partition k
    acc = np.empty((n_hidden, K))
    accTest = np.empty(n_hidden)
    i = 0
    for n in neurons: 
        start_it = time.time()
        k = 0
        for train_indices, valid_indices in kf.split(X):
            # Extract the training and validation sets from the K-fold split
            X_train_k = X_tensor[train_indices]
            y_train_k = l_tensor[train_indices]
            X_train_tensor = (X_train_k)
            y_train_tensor = (y_train_k)
            
            X_valid_k = X_tensor[valid_indices]
            y_valid_k = l_tensor[valid_indices]
            X_valid_tensor = (X_valid_k)
            
            
            #Train it
            model = TwoLayerMLP(input_dim, n, output_dim)
            model.to(device)
            optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
            criterion = nn.CrossEntropyLoss()
            num_epochs = 100
            model = model_train(model, X_train_tensor, y_train_tensor, criterion, optimizer, num_epochs=num_epochs)
            
            #Validate/predict it and record p(error)
            Z, acc[i,k], conf_matrix = model_predict(model,X_valid_tensor,y_valid_k)
#             Z, accTest[i,k] = model_predict(model,X_tensor_test,labelsTest)
            k += 1
        i+=1
#         print(i, 'done')

        stop_it = time.time()
        t = stop_it - start_it
#         print('Time to run {}th iteration: {}'.format(i,t)) 
        print('Time to run {}th iteration:'.format(i))
        print('{} minutes'.format(t/60)) 
        print()
    accuracy = np.mean(acc, axis=1)
    min_acc = np.min(accuracy)
    min_ind = np.argmin(accuracy)
    n_optimal = neurons[min_ind]
    
    return min_ind, min_acc, n_optimal, accuracy, acc#, models, accTest

In [8]:
print(device)
torch.cuda.device(device)

# return
input_dim = X.shape[1]
n_hidden_neurons = 16   #VARY THIS FOR CV
output_dim = C

# Convert data to pytorch tensors
X_tensor_train = torch.FloatTensor(X_train)
X_tensor_test = torch.FloatTensor(X_test)


l_tensor_train = torch.LongTensor(y_train)
l_tensor_test = torch.LongTensor(y_test)


print('done')
# Convert numpy structures to PyTorch tensors, as these are the data types required by the library
print('train:',len(X_train))
print('test: ',len(X_test))


cuda
done
train: 890723
test:  381739


In [9]:
#Keep track of time to check performance and runtime
start = time.time()

# start = 5
# neurons_test = np.arange(start, start+15, 1)
# neurons = np.zeros(len(Ntrain))
# prob_error = np.zeros(len(Ntrain))

# CV for best number of neurons
min_ind, min_acc, nNeurons,acc,acc_all = neuron_cross_validation(X_train,y_train,X_tensor_train,l_tensor_train)
# neurons = nNeurons
prob_error = min_acc
# print(Ntrain,"Samples...")


stop = time.time()
print('end')
print('Total Time (sec): ', stop - start) 
print('Total Time (min): ', (stop - start)/60)
print()
print("Best no. of neurons:            ",nNeurons)
print("Probability of error (Training):",min_acc)
print("Accuracy (Training)            :",1-min_acc)

Time to run 1th iteration:
0.4559074322382609 minutes

Time to run 2th iteration:
0.4384065667788188 minutes

Time to run 3th iteration:
0.44165541330973307 minutes

Time to run 4th iteration:
0.44390332301457724 minutes

Time to run 5th iteration:
0.445858895778656 minutes

Time to run 6th iteration:
0.45025063753128053 minutes

Time to run 7th iteration:
0.4598331054051717 minutes

Time to run 8th iteration:
0.4569467385609945 minutes

Time to run 9th iteration:
0.460508930683136 minutes

Time to run 10th iteration:
0.46407422224680583 minutes

Time to run 11th iteration:
0.4776325305302938 minutes

Time to run 12th iteration:
0.4781716108322144 minutes

Time to run 13th iteration:
0.473454745610555 minutes

Time to run 14th iteration:
0.47715186278025307 minutes

Time to run 15th iteration:
0.4786662658055623 minutes

Time to run 16th iteration:
0.4940929611523946 minutes

Time to run 17th iteration:
0.48810981909434 minutes

Time to run 18th iteration:
0.49332417249679567 minutes



KeyboardInterrupt: 

In [ ]:
print("Best no. of neurons:            ",nNeurons)
print("Probability of error (Training):",min_acc)
print("Accuracy (Training):",1-min_acc)
# print(acc)


# plt.scatter(neurons_test,acc)
# plt.show()

#Plot performance vs neuron count
start = 1
trials = 150
x_ax = np.arange(start, start+trials, 1)
plt.plot(x_ax,acc)
plt.title('Probability of Error vs. Neurons')
plt.ylabel('p(Error)')
plt.xlabel('Neurons')
plt.show()

In [ ]:
#Test Optimal number of neurons on test data
start = time.time()

model = TwoLayerMLP(input_dim, nNeurons, output_dim)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()
num_epochs = 100
model = model_train(model, X_tensor_train, l_tensor_train, criterion, optimizer, num_epochs=num_epochs)

#Validate/predict it and record p(error)
# y_test_tensor = Tensor
Z, accTest, conf_matrix = model_predict(model,X_tensor_test,l_tensor_test)

stop = time.time()

print('Probability of error with', nNeurons,'Neurons (Test):',accTest)
# print('Time: ', stop - start) 
print('Time (min): ', (stop - start)/60) 
print()


In [ ]:
print('Accuracy (Test):',1-accTest)
print('Worst Value    :',1/7)
print("Percent Per Class:")
print(df.index.value_counts().sort_index()/N*100)

In [ ]:
### Report Recall and Precision
#Precision and recall of model
print(conf_matrix)
for i in range(len(conf_matrix)):
    recall = np.mean(conf_matrix, axis = 0)
    precision = np.mean(conf_matrix, axis = 1)



start = time.time()

model = TwoLayerMLP(input_dim, nNeurons, output_dim)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()
num_epochs = 100
model = model_train(model, X_tensor_train, l_tensor_train, criterion, optimizer, num_epochs=num_epochs)
#Validate/predict it and record p(error)
Z, accTest = model_predict(model,X_tensor_test,y_test)

stop = time.time()

print('Probability of error with', nNeurons,'Neurons (Test):',accTest)
print('Time: ', stop - start) 
print('Time (min): ', (stop - start)/60) 

print('Accuracy (Test):',1-accTest)
print('Worst Value    :',1/7)

print(acc)